<a href="https://colab.research.google.com/github/Hao0827/2-1-Gwanabong/blob/main/%EC%A0%90%EC%88%98_%ED%95%A9%EC%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

"""colab_lv_widget.py – Adds bar chart of final populations each step
-----------------------------------------------------------------------
For each step, alongside the time‑series line plot, display a bar chart
of the *rounded* prey & predator populations at the current time.
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import ipywidgets as widgets
from IPython.display import display, clear_output

# -------- base constants --------
BASE_ALPHA = 1.0
BASE_BETA  = 0.1
BASE_GAMMA = 1.5
BASE_DELTA = 0.075

# -------- helpers ---------------
def make_piecewise(values, interval):
    return lambda t: values[min(int(t // interval), len(values) - 1)]

def lv_rhs(t, z, a, b, g, d):
    x, y = z
    return [a(t) * x - b(t) * x * y, d(t) * x * y - g(t) * y]

def simulate(t_end, y0, a_vals, b_vals, g_vals, d_vals, interval, n_points=2000):
    a, b, g, d = (
        make_piecewise(a_vals, interval),
        make_piecewise(b_vals, interval),
        make_piecewise(g_vals, interval),
        make_piecewise(d_vals, interval),
    )
    t_eval = np.linspace(0, t_end, n_points)
    sol = solve_ivp(lv_rhs, (0, t_end), y0, t_eval=t_eval, args=(a, b, g, d))
    return sol.t, sol.y[0], sol.y[1]

# ---- Phase 1: configuration ----
steps_box = widgets.IntText(value=10, description="Steps", style={"description_width": "60px"})
dur_box   = widgets.IntText(value=10, description="Δt",   style={"description_width": "60px"})
start_btn = widgets.Button(description="Start simulator", button_style="success")
cfg_msg   = widgets.HTML()
display(widgets.HBox([steps_box, dur_box, start_btn]), cfg_msg)

ui_container = widgets.VBox()
display(ui_container)

sim = {}  # state dict

def on_start(_):
    cfg_msg.value = ""
    N, dt = steps_box.value, dur_box.value
    if N <= 0 or dt <= 0:
        cfg_msg.value = "<span style='color:red'>Steps and Δt must be positive.</span>"
        return

    sim.update({
        "N": N, "dt": dt, "step": 0,
        "alpha": [], "beta": [], "gamma": [], "delta": [],
        "x0": 10.0, "y0": 5.0
    })

    # Widgets for percentage changes
    style = {"description_width": "40px"}
    pa = widgets.FloatText(value=0.0, description="α %", style=style, step=1)
    pb = widgets.FloatText(value=0.0, description="β %", style=style, step=1)
    pg = widgets.FloatText(value=0.0, description="γ %", style=style, step=1)
    pd = widgets.FloatText(value=0.0, description="δ %", style=style, step=1)
    next_btn = widgets.Button(description=f"Next (0/{N})", button_style="primary")
    status   = widgets.HTML()

    plot_out = widgets.Output()

    ui_container.children = [widgets.HBox([pa, pb, pg, pd, next_btn]), status, plot_out]

    sim.update({"boxes": (pa, pb, pg, pd), "btn": next_btn, "msg": status, "out": plot_out})

    def on_next(_):
        k = sim["step"]
        if k >= sim["N"]:
            return

        perc = [box.value for box in sim["boxes"]]
        a_val = BASE_ALPHA * (1 + perc[0] / 100)
        b_val = BASE_BETA  * (1 + perc[1] / 100)
        g_val = BASE_GAMMA * (1 + perc[2] / 100)
        d_val = BASE_DELTA * (1 + perc[3] / 100)

        sim["alpha"].append(a_val)
        sim["beta"].append(b_val)
        sim["gamma"].append(g_val)
        sim["delta"].append(d_val)
        sim["step"] += 1

        t_end = sim["step"] * sim["dt"]
        t, x, y = simulate(
            t_end,
            (sim["x0"], sim["y0"]),
            sim["alpha"], sim["beta"], sim["gamma"], sim["delta"],
            sim["dt"]
        )

        # Rounded populations
        prey_final = int(round(x[-1]))
        pred_final = int(round(y[-1]))

        # Plot line + bar chart
        with sim["out"]:
            clear_output(wait=True)
            fig, axs = plt.subplots(1, 2, figsize=(12, 4))
            # Line plot
            axs[0].plot(t, x, label="Prey (x)")
            axs[0].plot(t, y, label="Predator (y)")
            axs[0].set_xlim(0, sim["N"] * sim["dt"])
            axs[0].set_xlabel("Time")
            axs[0].set_ylabel("Population")
            axs[0].set_title(f"Lotka–Volterra (0–{t_end})")
            axs[0].grid(True)
            axs[0].legend()

            # Bar chart
            axs[1].bar(["Prey", "Predator"], [prey_final, pred_final])
            axs[1].set_ylabel("Count")
            axs[1].set_title(f"Populations at t={t_end}")
            for i, v in enumerate([prey_final, pred_final]):
                axs[1].text(i, v + max(v*0.05, 0.5), str(v), ha="center", va="bottom")

            plt.tight_layout()
            plt.show()

        # Update UI
        if sim["step"] < sim["N"]:
            sim["btn"].description = f"Next ({sim['step']}/{sim['N']})"
            sim["msg"].value = f"<b>Step {sim['step']}/{sim['N']} recorded.</b>"
        else:
            sim["btn"].description = "Done"
            sim["btn"].disabled = True
            sim["msg"].value = "<b>✅ All steps finished.</b>"

    next_btn.on_click(on_next)
    cfg_msg.value = f"<b>Simulator initialized for N={N}, Δt={dt}.</b>"

start_btn.on_click(on_start)
